In [1]:
import pandas as pd

In [ ]:
import pandas as pd

df = pd.read_parquet('../data/processed/dados_completos_commodities.parquet')

# Tratamento de Buracos (Missing Values) gerados por feriados
# Utilizamos ffill() (Forward Fill) para propagar o último preço conhecido
# Lógica: Se hoje é feriado em NY, o preço vale o mesmo de ontem.
df_tratado = df.ffill()

# Removemos apenas o início da série se não houver dados históricos suficientes para o preenchimento
df_tratado = df_tratado.dropna()

# Definição das constantes de conversão de unidades
FATOR_ARABICA = 132.276  # 1 Saca (60kg) = 132.276 Libras-peso (Unidade de NY)
FATOR_ROBUSTA = 0.06     # 1 Saca (60kg) = 0.06 Toneladas (Unidade de Londres)

# Criação das colunas de Preço Real em BRL (Reais por Saca de 60kg)
df_tratado['Arabica_BRL'] = (df_tratado['Arabica'] / 100) * FATOR_ARABICA * df_tratado['Dolar']

# Fórmula Robusta: Preço Tonelada * Fator Tonelada/Saca * Dolar
df_tratado['Robusta_BRL'] = df_tratado['Robusta'] * FATOR_ROBUSTA * df_tratado['Dolar']


df_tratado['Arabica_BRL'] = df_tratado['Arabica_BRL'].round(2)
df_tratado['Robusta_BRL'] = df_tratado['Robusta_BRL'].round(2)

# Salvando o arquivo enriquecido na pasta processed
df_tratado.to_parquet('../data/processed/dados_completos_commodities.parquet')

# --- VALIDAÇÃO ---
print("✅ Transformação concluída com sucesso!")
print("--- Amostra dos Dados Finais (R$/Saca) ---")
display(df_tratado[['Dolar', 'Arabica_BRL', 'Robusta_BRL']].tail())
df_tratado.info()

✅ Transformação concluída com sucesso!
--- Amostra dos Dados Finais (R$/Saca) ---


,Dolar,Arabica_BRL,Robusta_BRL
Date,,,
2026-01-12,5.3703,2529.24,1238.93
2026-01-13,5.3737,2560.70,1251.64
2026-01-14,5.3713,2529.36,1250.12
2026-01-15,5.3961,2556.03,1270.13
2026-01-16,5.3694,2522.07,1292.84


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1300 entries, 2021-01-19 to 2026-01-16
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Dolar        1300 non-null   float64
 1   Arabica      1300 non-null   float64
 2   Robusta      1300 non-null   float64
 3   Arabica_BRL  1300 non-null   float64
 4   Robusta_BRL  1300 non-null   float64
dtypes: float64(5)
memory usage: 60.9 KB
